In [1]:
from hardware.opentrons.http_communications import OpentronsAPI
from hardware.cameras import PendantDropCamera
from hardware.sensor.sensor_api import SensorAPI
from hardware.opentrons.configuration import Configuration

opentrons_api = OpentronsAPI()
opentrons_api.initialise()
config = Configuration(http_api=opentrons_api)
labware = config.load_labware()
containers = config.load_containers()
pipettes = config.load_pipettes()
left_pipette = pipettes["left"]
right_pipette = pipettes["right"]
# pendant_drop_camera = PendantDropCamera()

2025-02-28 16:21:12,312 - INFO - Protocol already uploaded, using existing protocol.
2025-02-28 16:21:19,449 - INFO - Run created succesfully (ID: f02926c6-409a-46f2-8a7e-a249907862b8).
2025-02-28 16:21:21,280 - INFO - All custom labware definitions added.
2025-02-28 16:21:22,099 - INFO - Labware loaded successfully.
2025-02-28 16:21:22,148 - INFO - Containers loaded successfully.
2025-02-28 16:21:26,266 - INFO - Pipettes loaded successfully.


In [2]:
opentrons_api.home()

2025-02-28 16:07:38,589 - INFO - Robot homed.


In [2]:
ft50 = containers["5A1"]
ft50.INNER_DIAMETER_MM

28